In [1]:
import pandas as pd

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,50,2024-02-04,2,111.571429,97.589796,0.530592,12.271429,25.975510,0.234000,113.614286,...,13.156250,25.106250,0.197437,112.595833,0,2,6,2,22,2024
1,50,2024-02-04,-1,116.877551,97.246939,0.567878,12.616327,25.004082,0.239592,119.616327,...,11.658333,19.639583,0.182208,112.156250,0,2,6,2,24,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', show=True):
    if model_name == 'LR':
        model = joblib.load('Models/TrainedModels/LR_Model.pkl')
        scaler = joblib.load('Models/Scalers/LR.bin')
    elif model_name == 'RF':
        model = joblib.load('Models/TrainedModels/RF_Model.pkl')
        scaler = joblib.load('Models/Scalers/RF.bin')
    elif model_name == 'GB':
        model = joblib.load('Models/TrainedModels/GB_Model.pkl')
        scaler = joblib.load('Models/Scalers/GB.bin')
    elif model_name == 'SVM':
        model = joblib.load('Models/TrainedModels/SVM_Model.pkl')
        scaler = joblib.load('Models/Scalers/SVM.bin')
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    predictions = model.predict(scaled_df)
    
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-04,ORL,DET,1.359712,3.25,1
1,2024-02-04,PHO,WAS,1.160000,5.55,1


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-04,ORL,DET,1.359712,3.25,0
1,2024-02-04,PHO,WAS,1.160000,5.55,1


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-04,ORL,DET,1.359712,3.25,0
1,2024-02-04,PHO,WAS,1.160000,5.55,1


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-02-04,ORL,DET,1.359712,3.25,0
1,2024-02-04,PHO,WAS,1.160000,5.55,1
